In [ ]:
# 記事反映
import requests
import json
import mysql
import mysql.connector
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import urllib.request as req
import glob
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
from dotenv import load_dotenv
from selenium.webdriver.common.by import By

from sqlalchemy import (Column, String, Text, ForeignKey, \
                create_engine, MetaData, DECIMAL, DATETIME, exc, event, Index)
from sqlalchemy.schema import UniqueConstraint
from sqlalchemy.orm import (sessionmaker, relationship, scoped_session)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.mysql import INTEGER as Integer
from datetime import datetime
from sqlalchemy import Column, Integer, create_engine, MetaData, DATETIME, String,Date,Float
from sqlalchemy.orm import sessionmaker
from pathlib import Path
import datetime
from sqlalchemy import create_engine, text
from  src.utils import *
print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()
class PledgeScraping():

    def add_target_date(self, target_day_number:int):
        week_list = [ '(日)','(月)', '(火)', '(水)', '(木)', '(金)', '(土)','(日)']
        target_date:datetime = datetime.date.today() + datetime.timedelta(days=target_day_number)
        #datetime(2023,3,3)
        self.target_date:datetime = datetime.date.today() + datetime.timedelta(days=target_day_number)
        #3月3日(火)
        self.target_date_string_jp:str = target_date.strftime('%m').lstrip('0') + '月' + target_date.strftime('%d').lstrip('0') + '日'  +week_list[target_date.isoweekday()]
        #2023-03-03
        self.target_date_string_sql:str = target_date.strftime('%Y-%m-%d')
        print(f'インスタンに日付:{self.target_date_string_jp}など三つの変数が追加されました')

    def login_scraping_site(self,area_name):
        global browser
        options = Options()
        options.add_argument('--headless')
        options.add_argument("--no-sandbox")
        browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install()
        browser.implicitly_wait(10)
        url_login = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/login_form_mail"
        #admageを開く
        browser.get(url_login)
        browser.implicitly_wait(10)
        # id
        element = browser.find_element(By.NAME,"id")
        element.click()
        element.clear()
        browser.implicitly_wait(10)
        element.send_keys(os.getenv('REPORT_SITE_ID'))
        # pw
        element = browser.find_element(By.NAME,"pass")
        element.click()
        element.clear()
        browser.implicitly_wait(10)
        element.send_keys(os.getenv('REPORT_SITE_PW'))
        browser.implicitly_wait(10)
        element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
        element.click()
        browser.implicitly_wait(10)
        url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/select_area"
        browser.get(url)
        browser.implicitly_wait(10)
        url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/?area={area_name}"
        browser.get(url)
        browser.implicitly_wait(10)
        self.browser =browser
        return self.browser

#エリアの都道府県の名前と対応番号を取得

    def get_prefecture_name_and_number_dict(self):
        prefectures_box_elements = browser.find_element(By.CLASS_NAME,"one_sp_block")
        prefectures_elements:list = prefectures_box_elements.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
        prefecture_name_and_number_dict:dict[str:str] = {}

        for element in prefectures_elements:
            prefecture_name:str = element.find_element(By.CLASS_NAME,"side_bar_L_O").text
            #print(prefecture_name)
            link_url:str = element.find_element(By.TAG_NAME,'a').get_attribute('href')
            prefecture_number:int = link_url.split('ken=')[-1]
            #print(prefecture_number)
            prefecture_name_and_number_dict[prefecture_name] = prefecture_number

        self.prefecture_name_and_number_dict = prefecture_name_and_number_dict
        return self.prefecture_name_and_number_dict
    
    def read_target_date_pledge_df(self,prefecture_name):
        browser = self.browser
        self.prefecture_name = prefecture_name
        furture_syuzai_list_df = pd.DataFrame(index=[], columns=['都道府県','イベント日','店舗名','取材名','取材ランク'])
        prefecture_number:int = int(self.prefecture_name_and_number_dict[prefecture_name])
        url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={self.prefecture_name}&ymd={self.target_date_string_sql}"
        browser.get(url)
        browser.implicitly_wait(10)
        kiji_element_box = browser.find_element(By.CLASS_NAME,"osusume_list_container")
        kiji_elements_list:list = kiji_element_box.find_elements(By.CLASS_NAME,"osbox")

        for i,syuzai_record_element in enumerate(kiji_elements_list):
            if 'プレミアム会員登録' == browser.find_element(By.CLASS_NAME,"menu_child").text:
                browser = self.login_scraping_site('chubu')
                url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={self.target_date_string_sql}&ymd={self.target_date_string_sql}"
                browser.get(url)
                browser.implicitly_wait(10)
            else:
                pass
            #print(syuzai_record_element.text)
            syuzai_record_element_list = syuzai_record_element.find_elements(By.CLASS_NAME,"list_event_name")
            browser.implicitly_wait(1)
            #print(syuzai_record_element_list)
            if len(syuzai_record_element_list) == 0:
                continue
            tenpo_name = syuzai_record_element.find_element(By.CLASS_NAME,"oslh2").text
            for syuzai_name_element in syuzai_record_element_list:
                #print(syuzai_name_element.text)
                syuzai_rank = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_rank").text
                schedule_name = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_li").text
                #print(tenpo_name,syuzai_rank,schedule_name)
                record = pd.Series([prefecture_name,self.target_date_string_sql, tenpo_name,schedule_name,syuzai_rank], index=furture_syuzai_list_df.columns)
                furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)

        self.furture_syuzai_list_df = furture_syuzai_list_df
        return self.furture_syuzai_list_df

    def read_convert_parlar_name_df(self):
        cnx = mysql.connector.connect(
                        user = os.getenv('DB_USER_NAME'),
                        password=os.getenv('DB_PASSWORD'), 
                        host=os.getenv('DB_HOST'), 
                        port='3306',
                        database=os.getenv('DB_NAME'))
        cursor = cnx.cursor()
        sql = f"""SELECT * FROM pledge"""
        print(sql)
        cursor.execute(sql)
        #cols = [col[0] for col in cursorsor.description]
        convert_parlar_name_df = pd.DataFrame(cursor.fetchall(),columns = ['id','取材名','媒体名','公約内容','取得時間'])
        self.convert_parlar_name_df = convert_parlar_name_df
        return self.convert_parlar_name_df
    
    def generate_merged_syuzai_pledge_df(self):
        merged_syuzai_pledge_df = pd.merge(self.furture_syuzai_list_df,self.convert_parlar_name_df,how='left',on='取材名')
        merged_syuzai_pledge_df = merged_syuzai_pledge_df[~merged_syuzai_pledge_df['取材名'].str.contains('ナビ子')]
        merged_syuzai_pledge_df = merged_syuzai_pledge_df.fillna('未調査')
        merged_syuzai_pledge_df = merged_syuzai_pledge_df.replace({'': '未調査'})
        self.merged_syuzai_pledge_df = merged_syuzai_pledge_df
        return self.merged_syuzai_pledge_df
    
    def recommend_image(self,write_image_context, length,read_image_path,save_image_jpg):
        image = Image.open(read_image_path)
        draw = ImageDraw.Draw(image)
        # フォントを指定
        font_path = r"font\LightNovelPOPv2.otf"
        font = ImageFont.truetype(font_path, size=40)

        # 文字を描く
        # 最初の(0,0)は文字の描画を開f始する座標位置です　もちろん、(10,10)などでもOK
        # fillはRGBで文字の色を決めています
        draw.multiline_text((55, 40), write_image_context, fill=(255, 255, 255), font=font, spacing=10, stroke_width=5, stroke_fill=(55, 55, 55))

        # 画像を保存する
        image.save(save_image_jpg)
        print('length', length)
        length_croped = 110 + length * 66  # 500
        im = Image.open(save_image_jpg)
        im_crop = im.crop((0, 0, 1000, length_croped))
        im_crop.save(save_image_jpg, quality=100)

    def create_pledge_main_images(self):
        read_image_path:str = r"image\image_source\eva_board.jpg"
        create_image_number:int = 0
        parlar_name_count:int = 0
        schedule_text:str = ''
        last_time_parlar_name:str = ''
        save_main_image_path_list:list[str] = []
        for i,row in merged_syuzai_pledge_df.iterrows():
            #print(row)
            print(row['店舗名'],last_time_parlar_name )
            if last_time_parlar_name != row['店舗名']:
                schedule_text += '\n◆' +  row['店舗名'] + '\n'
                parlar_name_count += 1
                #print(parlar_name_count)
            else:
                pass


            if row['取材名'].startswith(tuple(['旧イベ'])):
                schedule_text += '　☆' + row['取材名']  + '\n'
                last_time_parlar_name = row['店舗名']
                continue
            else:
                schedule_text += '　☆' + row['取材名'] + ' 【' + row['媒体名'] + '】' + '\n'


            if (str(row['公約内容']) == 'nan') or (row['公約内容'] == None) or (row['公約内容'] == ''):
                schedule_text += '       ┗' + '未調査' + '\n'
            else:
                schedule_text += '       ┗' + row['公約内容'] + '\n'

            last_time_parlar_name = row['店舗名']
            #print(parlar_name_count)
            if (parlar_name_count > 9 ) or merged_syuzai_pledge_df.index[-1] == i :
                #print('処理開始')
                create_image_number  += 1
                title = f'{prefecture_name} {scraping.target_date_string_jp} 明日のスロットイベントまとめ{create_image_number}\n'
                input_image_text : str = title + schedule_text
                text__row_length = input_image_text.count('\n')
                #print('length', text__row_length )
                print(input_image_text)
                save_image_jpg = fr"image\temp_image\eva_board_{self.prefecture_name}_{self.target_date_string_sql}_{create_image_number}.jpg"
                self.recommend_image(input_image_text, text__row_length ,read_image_path,save_image_jpg)
                save_main_image_path_list.append(save_image_jpg)
                schedule_text = ''
                last_time_parlar_name = ''
                parlar_name_count = 0
        self.save_main_image_path_list = save_main_image_path_list
        return self.save_main_image_path_list


scraping = PledgeScraping()
scraping.add_target_date(1)
browser = scraping.login_scraping_site('chubu')
prefecture_name_and_number_dict = scraping.get_prefecture_name_and_number_dict()
for prefecture_name in prefecture_name_and_number_dict:
    print(prefecture_name)
    content_html = ''
    prefecture_name_and_number_dict
    scraping_target_date_pledge_df = scraping.read_target_date_pledge_df(prefecture_name)
    read_convert_parlar_name_df = scraping.read_convert_parlar_name_df()
    merged_syuzai_pledge_df = scraping.generate_merged_syuzai_pledge_df()
    save_main_image_path_list = scraping.create_pledge_main_images()
    break




In [2]:
save_main_image_path_list

['image\\temp_image\\eva_board_愛知県_2023-03-27_1.jpg',
 'image\\temp_image\\eva_board_愛知県_2023-03-27_2.jpg',
 'image\\temp_image\\eva_board_愛知県_2023-03-27_3.jpg']

In [31]:
# def query_
# username = os.getenv('DB_USER_NAME')
# password=os.getenv('DB_PASSWORD')
# host=os.getenv('DB_HOST')
# port='3306'
# database=os.getenv('DB_NAME')
# db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"

# # DB接続するためのEngineインスタンス
# engine = create_engine(db_url, echo=False)

# # Sessionの作成
# session = scoped_session(sessionmaker(autocommit = False,autoflush = False,bind = engine))
# # modelで使用する
# Base = declarative_base()
# Base.query = session.query_property()





{'tomorrow_8numbers_string': '20230327', 'tomorrow_date_stinrg_jp': '3/27(月)', 'tomorrow_date_string_sql': '2023-03-27', 'yesterday': datetime.date(2023, 3, 25), 'tomorrow': datetime.date(2023, 3, 27), 'yesterday_date_string': '3/25(土)', 'id': '', 'pw': '', 'tweet_text': '', 'image_path_list': [], 'main_tweet_text': '', 'project_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site', 'image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image', 'tweet_footer_text': '\n\n#XXXX 20230325\n\n', 'created_image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image\\image', 'servise_account_json_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\config\\twitteranalytics-jsonsercretkey.json'}


In [3]:
html_table_df = merged_syuzai_pledge_df[['イベント日','店舗名','取材名','媒体名','公約内容']]
html_table_df['イベント日'] = scraping.target_date_string_jp
html_table_df = html_table_df[['イベント日','店舗名','取材名','媒体名','公約内容']]
print(html_table_df.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>イベント日</th>
      <th>店舗名</th>
      <th>取材名</th>
      <th>媒体名</th>
      <th>公約内容</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>3月27日(月)</td>
      <td>キクヤ春日井店</td>
      <td>ガチアンケート金色</td>
      <td>未調査</td>
      <td>未調査</td>
    </tr>
    <tr>
      <td>3月27日(月)</td>
      <td>キクヤ春日井店</td>
      <td>GPL</td>
      <td>アツ姫</td>
      <td>3台以上構成機種の3機種以上に全456\r\n　　　+総台数の10％に設定6を投入\r\n　　　+3台以上設置機種の1機種に全6\r\n　　　+3か所に5台並びで設定全56</td>
    </tr>
    <tr>
      <td>3月27日(月)</td>
      <td>マルハン新守山駅前店</td>
      <td>アドリブスロッター</td>
      <td>パチンコ店長のホール攻略</td>
      <td>未調査</td>
    </tr>
    <tr>
      <td>3月27日(月)</td>
      <td>メガコンコルド1177大口41号通り店</td>
      <td>サプライズスロッター</td>
      <td>パチンコ店長のホール攻略</td>
      <td>未調査</td>
    </tr>
    <tr>
      <td>3月27日(月)</td>
      <td>メガコンコルド1177大口41号通り店</td>
      <td>プレミアム×パチンカー</td>
      <td>パチンコ店長のホール攻略</td>
      <td>未調査</td>
    </tr>
    <tr>


C:\Users\tsc95\AppData\Local\Temp\ipykernel_39908\3028997999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  html_table_df['イベント日'] = scraping.target_date_string_jp
